In [1]:
import pandas as pd

In [2]:
# df = pd.read_csv("/kaggle/input/deeplearing-project/top_10_tags_encoded.csv")
df = pd.read_csv('./finalData/top_10_tags_encoded.csv', encoding='latin1')

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras.callbacks import EarlyStopping

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [3]:
# Parameters
MAX_NUM_WORDS = 10000
MAX_SEQUENCE_LENGTH = 300
# EMBEDDING_DIM = 100
# EPOCHS = 5
# BATCH_SIZE = 128

In [6]:
# Tokenize
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(df['clean_text'])
sequences = tokenizer.texts_to_sequences(df['clean_text'])

In [8]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [9]:
# Pad
X_seq = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [ ]:
# Labels (one-hot multilabel)
y = df.iloc[:, 1:].values  # clean_text is the first column

In [11]:
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report

In [12]:
model = load_model("./finalData/final_model.h5")

In [13]:
# Predict probabilities
y_pred_probs = model.predict(X_seq)

22054/22054 ━━━━━━━━━━━━━━━━━━━━ 1703s 77ms/step


In [14]:
# Apply best thresholds (from earlier tuning)
best_thresholds = np.array([0.55, 0.45, 0.40, 0.35, 0.55, 0.55, 0.45, 0.50, 0.45, 0.60])
y_pred_final = (y_pred_probs > best_thresholds).astype(int)

In [15]:
# Ground truth
y_true = y  # already used in training

In [16]:
# Classification Report
label_names = ['javascript', 'java', 'c#', 'php', 'android', 'jquery', 'python', 'html', 'c++', 'ios']
print(classification_report(y_true, y_pred_final, target_names=label_names))

              precision    recall  f1-score   support

  javascript       0.98      0.95      0.97     90588
        java       0.93      0.95      0.94    101077
          c#       0.96      0.92      0.94     47544
         php       0.72      0.68      0.70     58970
     android       0.98      0.95      0.97     46976
      jquery       0.97      0.84      0.90    115011
      python       0.86      0.82      0.84    124071
        html       0.86      0.84      0.85     78526
         c++       0.95      0.95      0.95     98750
         ios       0.99      0.97      0.98     64553

   micro avg       0.92      0.89      0.90    826066
   macro avg       0.92      0.89      0.90    826066
weighted avg       0.92      0.89      0.90    826066
 samples avg       0.93      0.92      0.92    826066



/Users/meshach/miniconda3/envs/pythonEnv2Mesh/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
